<a href="https://colab.research.google.com/github/Mohammad-Amin-Jenadele/gnn-graph-classification/blob/dev/GNN_Graph_classification_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries and packages

In [1]:
! pip install torch_geometric ogb torch==2.5.1 torchvision torchaudio --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 701.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.2 MB/s eta 0:00:00

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from ogb.graphproppred import PygGraphPropPredDataset
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
from torch_geometric.nn import global_mean_pool, global_add_pool
from torch_geometric.nn import GCNConv, GINEConv, GATv2Conv
from torch_geometric.utils import to_networkx
from torch_geometric.loader import DataLoader
import torch_geometric

# Setting configs and hyperparameters

In [3]:
# Choosing device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

device: cpu


# Data loading and preprocessing

# 📘 OGB MolHIV Dataset (Raw)

The dataset **ogbg-molhiv** consists of **41,127 graphs (molecules)**.  
Each molecule is labeled **0 or 1**, indicating whether it inhibits HIV replication (**1 = active**, **0 = inactive**).

---

## 🔹 Graph structure

- **Nodes = atoms**  
  Each node has **9 features**, representing atom properties such as:
  - Atomic number  
  - Formal charge  
  - Aromaticity  
  - Chirality  
  - Degree  
  - ... and more  

- **Edges = bonds**  
  Each edge has **3 features**, representing bond properties such as:
  - Bond type (single, double, aromatic, etc.)  
  - Bond stereochemistry  
  - Whether the bond is conjugated  

---

## 🔹 Raw CSV files

### 1. `node-feat.csv.gz`
- Shape: **(total_nodes, 9)**  
- Each row = **one atom (node)**  
- Columns = numerical features describing the atom (atomic number, chirality, formal charge, hybridization, aromaticity, etc.)  
- The rows are grouped by graph: first all nodes of graph 0, then graph 1, etc.  

---

### 2. `edge-feat.csv.gz`
- Shape: **(total_edges, 3)**  
- Each row = **one bond (edge)**  
- Columns = numerical features describing the bond:  
  - Bond type (single/double/triple/aromatic)  
  - Bond stereochemistry (e.g., cis/trans)  
  - Conjugation flag (0/1)  
- The rows are aligned with `edge.csv.gz` (row *i* in `edge-feat.csv.gz` corresponds to edge *i* in `edge.csv.gz`).  

---

### 3. `edge.csv.gz`
- Shape: **(total_edges, 2)**  
- Each row = **a directed edge** represented as `[src, dst]` (node indices).  
- Since bonds are undirected, each bond appears **twice**: once as `[u, v]` and once as `[v, u]`.  
- Indices are **local to the graph** (reset per molecule).  

---

### 4. `graph-label.csv.gz`
- Shape: **(num_graphs, 1)**  
- Each row = binary label (0 = inactive, 1 = active).  
- Tells whether the molecule inhibits HIV replication.  

---

### 5. `num-node-list.csv.gz`
- Shape: **(num_graphs, 1)**  
- Each row = number of nodes (atoms) in that graph (molecule).  
- Used to split the long `node-feat.csv.gz` into per-graph chunks.  

---

### 6. `num-edge-list.csv.gz`
- Shape: **(num_graphs, 1)**  
- Each row = number of edges (bonds) in that graph (molecule).  
- Used to split the long `edge.csv.gz` and `edge-feat.csv.gz` into per-graph chunks.  

---


In [5]:
# Load the dataset
dataset = PygGraphPropPredDataset(name='ogbg-molhiv' , root = '/content/')

/usr/local/lib/python3.12/dist-packages/ogb/graphproppred/dataset_pyg.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed

In [9]:
# preprocessing the raw data
edge_feat_df = pd.read_csv('/content/ogbg_molhiv/raw/edge-feat.csv.gz' , compression = 'gzip', header = None)
edge_df = pd.read_csv('/content/ogbg_molhiv/raw/edge.csv.gz' , compression = 'gzip', header = None)
graph_label_df = pd.read_csv('/content/ogbg_molhiv/raw/graph-label.csv.gz' , compression = 'gzip', header = None)
node_feat_df = pd.read_csv('/content/ogbg_molhiv/raw/node-feat.csv.gz' , compression = 'gzip', header = None)
num_edge_list_df = pd.read_csv('/content/ogbg_molhiv/raw/num-edge-list.csv.gz' , compression = 'gzip', header = None)
num_node_list_df = pd.read_csv('/content/ogbg_molhiv/raw/num-node-list.csv.gz' , compression = 'gzip', header = None)

In [20]:
edge_feat_df

,0,1,2
0,0,0,0
1,0,0,0
2,1,0,0
3,0,0,0
4,0,0,0
...,...,...,...
1129683,3,0,1
1129684,3,0,1
1129685,3,0,1
1129686,3,0,1


In [21]:
edge_df

,0,1
0,0,1
1,1,2
2,2,3
3,3,4
4,4,5
...,...,...
1129683,29,36
1129684,21,7
1129685,36,22
1129686,20,15


In [22]:
graph_label_df

,0
0,0
1,0
2,0
3,0
4,0
...,...
41122,0
41123,0
41124,0
41125,0


In [23]:
node_feat_df

,0,1,2,3,4,5,6,7,8
0,5,0,4,5,3,0,2,0,0
1,5,0,4,5,2,0,2,0,0
2,5,0,3,5,0,0,1,0,1
3,7,0,2,6,0,0,1,0,1
4,28,0,4,2,0,0,5,0,1
...,...,...,...,...,...,...,...,...,...
1049158,5,0,3,5,0,0,1,1,1
1049159,5,0,4,5,3,0,2,0,0
1049160,6,0,2,5,0,0,1,1,1
1049161,7,0,2,5,0,0,1,1,1


In [24]:
num_edge_list_df

,0
0,20
1,44
2,24
3,25
4,9
...,...
41122,32
41123,42
41124,38
41125,38


In [25]:
num_node_list_df

,0
0,19
1,39
2,21
3,24
4,10
...,...
41122,28
41123,36
41124,33
41125,33


In [26]:
num_node_list_df[0].sum()

np.int64(1049163)

In [27]:
num_edge_list_df[0].sum()

np.int64(1129688)